## Week 4, Lab 2: Predicting Chronic Kidney Disease in Patients
> Author: Matt Brems

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

In this lab, we're going to focus on steps exploring data, building models and evaluating the models we build.

There are three links you may find important:
- [A set of chronic kidney disease (CKD) data and other biological factors](./chronic_kidney_disease_full.csv).
- [The CKD data dictionary](./chronic_kidney_disease_header.txt).
- [An article comparing the use of k-nearest neighbors and support vector machines on predicting CKD](./chronic_kidney_disease.pdf).

## Step 1: Define the problem.

Suppose you're working for Mayo Clinic, widely recognized to be the top hospital in the United States. In your work, you've overheard nurses and doctors discuss test results, then arrive at a conclusion as to whether or not someone has developed a particular disease or condition. For example, you might overhear something like:

> **Nurse**: Male 57 year-old patient presents with severe chest pain. FDP _(short for fibrin degradation product)_ was elevated at 13. We did an echo _(echocardiogram)_ and it was inconclusive.

> **Doctor**: What was his interarm BP? _(blood pressure)_

> **Nurse**: Systolic was 140 on the right; 110 on the left.

> **Doctor**: Dammit, it's an aortic dissection! Get to the OR _(operating room)_ now!

> _(intense music playing)_

In this fictitious but [Shonda Rhimes-esque](https://en.wikipedia.org/wiki/Shonda_Rhimes#Grey's_Anatomy,_Private_Practice,_Scandal_and_other_projects_with_ABC) scenario, you might imagine the doctor going through a series of steps like a [flowchart](https://en.wikipedia.org/wiki/Flowchart), or a series of if-this-then-that steps to diagnose a patient. The first steps made the doctor ask what the interarm blood pressure was. Because interarm blood pressure took on the values it took on, the doctor diagnosed the patient with an aortic dissection.

Your goal, as a research biostatistical data scientist at the nation's top hospital, is to develop a medical test that can improve upon our current diagnosis system for [chronic kidney disease (CKD)](https://www.mayoclinic.org/diseases-conditions/chronic-kidney-disease/symptoms-causes/syc-20354521).

**Real-world problem**: Develop a medical diagnosis test that is better than our current diagnosis system for CKD.

**Data science problem**: Develop a medical diagnosis test that reduces both the number of false positives and the number of false negatives.

---

## Step 2: Obtain the data.

### 1. Read in the data.

In [64]:
#Importing the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix

In [5]:
kidney_solution_df =  pd.read_csv('chronic_kidney_disease_full.csv')
kidney_solution_df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


In [6]:
kidney_solution_df.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu',
       'sc', 'sod', 'pot', 'hemo', 'pcv', 'wbcc', 'rbcc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'class'],
      dtype='object')

### Data Dictionary

| Predictor | Description             |
|-----------|-------------------------|
| age       | age                     |
| bp        | blood pressure          |
| sg        | specific gravity        |
| al        | albumin                 |
| su        | sugar                   |
| rbc       | red blood cells         |
| pc        | pus cell                |
| pcc       | pus cell clumps         |
| ba        | bacteria                |
| bgr       | blood glucose random    |
| bu        | blood urea              |
| sc        | serum creatinine        |
| sod       | sodium                  |
| pot       | potassium               |
| hemo      | hemoglobin              |
| pcv       | packed cell volume      |
| wc        | white blood cell count  |
| rc        | red blood cell count    |
| htn       | hypertension            |
| dm        | diabetes mellitus       |
| cad       | coronary artery disease |
| appet     | appetite                |
| pe        | pedal edema             |
| ane       | anemia                  |
| class     | CKD or not              |

### 2. Check out the data dictionary. What are a few features or relationships you might be interested in checking out?

1) Relationship between artery disease and CKD

2) Sodium content and CKD

3) Anemia and CKD

---

## Step 3: Explore the data.

### 3. How much of the data is missing from each column?

In [8]:
kidney_solution_df.isnull().sum()

age        9
bp        12
sg        47
al        46
su        49
rbc      152
pc        65
pcc        4
ba         4
bgr       44
bu        19
sc        17
sod       87
pot       88
hemo      52
pcv       71
wbcc     106
rbcc     131
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
class      0
dtype: int64

In [ ]:
kidney_solution_df.dropn

### 4. Suppose that I dropped every row that contained at least one missing value. (In the context of analysis with missing data, we call this a "complete case analysis," because we keep only the complete cases!) How many rows would remain in our dataframe? What are at least two downsides to doing this?

> There's a good visual on slide 15 of [this deck](https://liberalarts.utexas.edu/prc/_files/cs/Missing-Data.pdf) that shows what a complete case analysis looks like if you're interested.

In [12]:
comp_case = kidney_solution_df.dropna()
print(f'The original dataframe consists of {len(kidney_solution_df)} rows')
print(f'Rows remaining in our dataframe for complete case analysis is {len(comp_case)}')

The original dataframe consists of 400 rows
Rows remaining in our dataframe for complete case analysis is 158


Downsides:

1) Data for running machine learning models would reduce leading to High Bias or High Variance.

2) Crucial data for other predictors in the dropped rows would be lost.

### 5. Thinking critically about how our data were gathered, it's likely that these records were gathered by doctors and nurses. Brainstorm three potential areas (in addition to the missing data we've already discussed) where this data might be inaccurate or imprecise.

1) Outlier Detection

2) Self reported data

3) Investigate more on distributions

---

## Step 4: Model the data.

### 6. Suppose that I want to construct a model where no person who has CKD will ever be told that they do not have CKD. What (very simple, no machine learning needed) model can I create that will never tell a person with CKD that they do not have CKD?

> Hint: Don't think about `statsmodels` or `scikit-learn` here.

False Positive: Someone told they have CKD, but they do not.


False Negative: Someone told they do not have CKD, but they do.



Answer: Model: Tell everyone that they might have CKD.

### 7. In problem 6, what common classification metric did we optimize for? Did we minimize false positives or negatives?

We optimised for False Negatives and avoided it completely.

### 8. Thinking ethically, what is at least one disadvantage to the model you described in problem 6?

Costs of unnecesscary tests & Medication for people with no diseases. Especially hard onpeople who do not have health coverage.

### 9. Suppose that I want to construct a model where a person who does not have CKD will ever be told that they do have CKD. What (very simple, no machine learning needed) model can I create that will accomplish this?

Tell everyone that they do not have CKD.

### 10. In problem 9, what common classification metric did we optimize for? Did we minimize false positives or negatives?

We optimised for False Positives and avoided it completely.

### 11. Thinking ethically, what is at least one disadvantage to the model you described in problem 9?

People who do have the disease will be wrongly told that they do not. Which is completely ethically wrong.

You are letting people who have the disease suffer on without treatment. Kidney Disorder is not a fun game.

I hate both these models and they belong in the trash. 

### 12. Construct a logistic regression model in `sklearn` predicting class from the other variables. You may scale, select/drop, and engineer features as you wish - build a good model! Make sure, however, that you include at least one categorical/dummy feature and at least one quantitative feature.

> Hint: Remember to do a train/test split!

In [16]:
#Checking the data
miss_columns = kidney_solution_df.isnull().sum()[kidney_solution_df.isnull().sum()>=80].index
print(miss_columns)

kidney_solution_df.drop(columns = miss_columns, inplace = True)
kidney_solution_df.dtypes

Index(['rbc', 'sod', 'pot', 'wbcc', 'rbcc'], dtype='object')


age      float64
bp       float64
sg       float64
al       float64
su       float64
pc        object
pcc       object
ba        object
bgr      float64
bu       float64
sc       float64
hemo     float64
pcv      float64
htn       object
dm        object
cad       object
appet     object
pe        object
ane       object
class     object
dtype: object

In [20]:
comp_case.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wbcc,rbcc,htn,dm,cad,appet,pe,ane,class
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,...,29.0,12100.0,3.7,yes,yes,no,poor,no,yes,ckd
11,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,380.0,...,32.0,4500.0,3.8,yes,yes,no,poor,yes,no,ckd
14,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,157.0,...,16.0,11000.0,2.6,yes,yes,yes,poor,yes,no,ckd
20,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,173.0,...,24.0,9200.0,3.2,yes,yes,yes,poor,yes,yes,ckd


In [22]:
#Checking null values
comp_case.isnull().sum().sum()

0

In [26]:
#Getting a list of categorical Columns
cat_cols = []
for i in comp_case.columns:
    if comp_case[i].dtype == 'object':
        cat_cols.append(i)
print (f'The number of categorical columns in the dataset is {len(cat_cols)}')

#Creating dummy variables
dummydrop = pd.get_dummies(comp_case[cat_cols], drop_first= True)
print (f'The number of dummy variables created is {len(dummydrop.columns)}')


#Dropping categorical variables from the original dataframe
comp_case.drop(cat_cols, 1, inplace = True)

#Creating the final complete data after dummy creation
preproc = pd.concat([comp_case, dummydrop], 1)
preproc.dropna(inplace =True)

The number of categorical columns in the dataset is 11
The number of dummy variables created is 11


/Users/antonypaulson/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [35]:
preproc['ckd_present'] = 1 - preproc['class_notckd']
preproc.drop('class_notckd',1, inplace = True)

In [37]:
preproc['ckd_present'].value_counts()

0    115
1     43
Name: ckd_present, dtype: int64

In [44]:
features = preproc.columns[:-1]
X = preproc[features]
y = preproc['ckd_present']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

logreg = LogisticRegression()
cross_score = cross_val_score(logreg, X_train, y_train, cv =5).mean()
print(cross_score)
logreg.fit(X_train, y_train)

0.9913043478260869


/Users/antonypaulson/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/antonypaulson/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/antonypaulson/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/antonypaulson/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/antonypaulson/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

---

## Step 5: Evaluate the model.

### 13. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your quantitative features.

In [60]:
preproc.columns

Index(['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo',
       'pcv', 'wbcc', 'rbcc', 'rbc_normal', 'pc_normal', 'pcc_present',
       'ba_present', 'htn_yes', 'dm_yes', 'cad_yes', 'appet_poor', 'pe_yes',
       'ane_yes', 'ckd_present'],
      dtype='object')

In [61]:
logreg.coef_[0]

0.045149638760167914

In [63]:
coef_dict = {preproc.columns[k]: logreg.coef_[0][k] for k in range(len(preproc.columns)-1)}
coef_dict

{'age': 0.045149638760167914,
 'bp': 0.12476618603899974,
 'sg': -0.0006089696807673807,
 'al': 0.0513746719059421,
 'su': 0.006012685743718475,
 'bgr': 0.08251846653440337,
 'bu': 0.15385545558807634,
 'sc': 0.039084141058432394,
 'sod': -0.2577309730855277,
 'pot': -0.009825338090886154,
 'hemo': -0.05863915425984504,
 'pcv': -0.2045298099670658,
 'wbcc': 0.0013768068513019612,
 'rbcc': -0.015104656587322964,
 'rbc_normal': -0.010860870043051009,
 'pc_normal': -0.007620133169904624,
 'pcc_present': 0.005336452534485181,
 'ba_present': 0.002593282658850127,
 'htn_yes': 0.007000639260230749,
 'dm_yes': 0.002140682200483976,
 'cad_yes': 0.00017934660024748704,
 'appet_poor': 0.004525221589536156,
 'pe_yes': 0.006279923063398696,
 'ane_yes': 0.005244349364931038}

The coefficient for age is 0.0451. This means that with each unit increase in age the logarithm of odds of CKD increases by 0.0451

### 14. Based on your logistic regression model constructed in problem 12, interpret the coefficient of one of your categorical/dummy features.

The coefficient for pcc present is 0.00533. This means that with pcc present vs absent, the logarithm of odds of CKD increases by 0.00533

### 15. Despite being a relatively simple model, logistic regression is very widely used in the real world. Why do you think that's the case? Name at least two advantages to using logistic regression as a modeling technique.

It is easy to interpret the effect of coefficients on the predicted variables.
It is unlike other classification methods which use Euclidean or other distance measures. It works using the sigmoid function.

### 16. Does it make sense to generate a confusion matrix on our training data or our test data? Why? Generate it on the proper data.

> Hint: Once you've generated your predicted $y$ values and you have your observed $y$ values, then it will be easy to [generate a confusion matrix using sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

It makes sense to create a confusion matrix on the test data. This is data on which the model has not been trained.

In [67]:
y_pred_test = logreg.predict(X)
pd.DataFrame(confusion_matrix(y_test, y_pred_test), 
             columns = ['Predicted Negative', 'Predicted Positive'], 
             index = ['Actual Negative', 'Actual Positive'])


,Predicted Negative,Predicted Positive
Actual Negative,30,0
Actual Positive,1,9


### 17. In this hospital case, we want to predict CKD. Do we want to optimize for sensitivity, specificity, or something else? Why? (If you don't think there's one clear answer, that's okay! There rarely is. Be sure to defend your conclusion!)

I would want to optimize for False negatives, cases where there is CKD but predicted negatives. That is I would optimize for sensitivity.


### 18 (BONUS). Write a function that will create an ROC curve for you, then plot the ROC curve.

Here's a strategy you might consider:
1. In order to even begin, you'll need some fit model. Use your logistic regression model from problem 12.
2. We want to look at all values of your "threshold" - that is, anything where .predict() gives you above your threshold falls in the "positive class," and anything that is below your threshold falls in the "negative class." Start the threshold at 0.
3. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
4. Increment your threshold by some "step." Maybe set your step to be 0.01, or even smaller.
5. At this value of your threshold, calculate the sensitivity and specificity. Store these values.
6. Repeat steps 3 and 4 until you get to the threshold of 1.
7. Plot the values of sensitivity and 1 - specificity.

In [68]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()

In [73]:
pred_proba = [i[1] for i in logreg.predict_proba(X_test)]

pred_df = pd.DataFrame({'true_values': y_test,
                        'pred_probs':pred_proba})

In [76]:
# Create figure.
plt.figure(figsize = (10,7))

# Create threshold values.
thresholds = np.linspace(0, 1, 200)

# Define function to calculate sensitivity. (True positive rate.)
def TPR(df, true_col, pred_prob_col, threshold):
    true_positive = df[(df[true_col] == 1) & (df[pred_prob_col] >= threshold)].shape[0]
    false_negative = df[(df[true_col] == 1) & (df[pred_prob_col] < threshold)].shape[0]
    return true_positive / (true_positive + false_negative)

# Define function to calculate 1 - specificity. (False positive rate.)
def FPR(df, true_col, pred_prob_col, threshold):
    true_negative = df[(df[true_col] == 0) & (df[pred_prob_col] <= threshold)].shape[0]
    false_positive = df[(df[true_col] == 0) & (df[pred_prob_col] > threshold)].shape[0]
    return 1 - (true_negative / (true_negative + false_positive))
    
# Calculate sensitivity & 1-specificity for each threshold between 0 and 1.
tpr_values = [TPR(y_pred_test, 'true_values', 'pred_probs', prob) for prob in thresholds]
fpr_values = [FPR(pred_df, 'true_values', 'pred_probs', prob) for prob in thresholds]

# Plot ROC curve.
plt.plot(fpr_values, # False Positive Rate on X-axis
         tpr_values, # True Positive Rate on Y-axis
         label='ROC Curve')

# Plot baseline. (Perfect overlap between the two populations.)
plt.plot(np.linspace(0, 1, 200),
         np.linspace(0, 1, 200),
         label='baseline',
         linestyle='--')

# Label axes.
plt.title('Receiver Operating Characteristic Curve', fontsize=22)
plt.ylabel('Sensitivity', fontsize=18)
plt.xlabel('1 - Specificity', fontsize=18)

# Create legend.
plt.legend(fontsize=16);

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

<Figure size 720x504 with 0 Axes>

### 19. Suppose you're speaking with the biostatistics lead at Mayo Clinic, who asks you "Why are unbalanced classes generally a problem? Are they a problem in this particular CKD analysis?" How would you respond?

Unbalanced classes are definitely a problem as the model is trained to predict only a certain class with more accuracy. Yes this is a problem in the ckd analysis. The model is trained to predict the non occurence of ckd than it is to predict occurence of CKD.

### 20. Suppose you're speaking with a doctor at Mayo Clinic who, despite being very smart, doesn't know much about data science or statistics. How would you explain why unbalanced classes are generally a problem to this doctor?

Unbalanced classes are definitely a problem as the model is trained to predict only a certain class with more accuracy. Yes this is a problem in the ckd analysis. The model is trained to predict the non occurence of ckd than it is to predict occurence of CKD.

### 21. Let's create very unbalanced classes just for the sake of this example! Generate very unbalanced classes by [bootstrapping](http://stattrek.com/statistics/dictionary.aspx?definition=sampling_with_replacement) (a.k.a. random sampling with replacement) the majority class.

1. The majority class are those individuals with CKD.
2. Generate a random sample of size 200,000 of individuals who have CKD **with replacement**. (Consider setting a random seed for this part!)
3. Create a new dataframe with the original data plus this random sample of data.
4. Now we should have a dataset with around 200,000 observations, of which only about 0.00075% are non-CKD individuals.

In [101]:
from sklearn.utils import resample
boot = resample(preproc, n_samples =200000, random_state=42)
boot.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_poor,pe_yes,ane_yes,ckd_present
337,44.0,70.0,1.025,0.0,0.0,92.0,40.0,0.9,141.0,4.9,...,1,0,0,0,0,0,0,0,0,0
320,57.0,60.0,1.020,0.0,0.0,105.0,49.0,1.2,150.0,4.7,...,1,0,0,0,0,0,0,0,0,0
91,56.0,70.0,1.015,4.0,1.0,210.0,26.0,1.7,136.0,3.8,...,1,0,0,0,0,0,0,0,0,1
341,63.0,70.0,1.025,0.0,0.0,130.0,37.0,0.9,150.0,5.0,...,1,0,0,0,0,0,0,0,0,0
284,33.0,80.0,1.025,0.0,0.0,100.0,37.0,1.2,142.0,4.0,...,1,0,0,0,0,0,0,0,0,0


### 22. Build a logistic regression model on the unbalanced class data and evaluate its performance using whatever method(s) you see fit. How would you describe the impact of unbalanced classes on logistic regression as a classifier?
> Be sure to look at how well it performs on non-CKD data.

---

## Step 6: Answer the problem.

At this step, you would generally answer the problem! In this situation, you would likely present your model to doctors or administrators at the hospital and show how your model results in reduced false positives/false negatives. Next steps would be to find a way to roll this model and its conclusions out across the hospital so that the outcomes of patients with CKD (and without CKD!) can be improved!